In [5]:
import pandas as pd

df = pd.read_json("../data/preprocessed/covid_test")
df

,text,label,hashtags,emojis,polarity,subjectivity,sentiment
0,post claim compulsory vaccination violates pri...,0,[],[trade_mark],0.033333,0.500000,positive
1,photo claim person doctor died attending covid...,0,[],[],0.500000,0.500000,positive
2,post video claim protest confiscation town ama...,0,[],[],0.000000,0.000000,neutral
3,death respiratory failure pneumonia registered...,0,[],[],-0.316667,0.300000,negative
4,dean college biologist adieu state lot cpr fal...,0,[],[trade_mark],-0.400000,0.600000,negative
5,household covid patient porto allegra campos g...,0,[],[],0.000000,0.000000,neutral
6,chain list recommendation prevent treat corona...,0,[],[],0.000000,0.000000,neutral
7,60000 argentinian company closed due covid com...,0,[],[],-0.112500,0.237500,negative
8,social medium post criticize photo smiling pre...,0,[],[],0.033333,0.066667,positive
9,cdc released update coronavirus transmitted,0,[],[],0.000000,0.000000,neutral


In [8]:
X = df["text"]
y = df["label"]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [13]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to C:/Users/Yaroslav/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

c:\users\yaroslav\appdata\local\programs\python\python38\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yaroslav\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [20]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [21]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

In [47]:
dataset = dataset.map(tokenize_dataset)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [49]:
tf_dataset = model.prepare_tf_dataset(
    dataset["train"], batch_size=16, shuffle=True, tokenizer=tokenizer
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [50]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(3e-5))
model.fit(tf_dataset)  # doctest: +SKIP

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
533/533 [==============================] - 995s 2s/step - loss: 0.4136


In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})

In [45]:
dataset["train"][0]["text"]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [42]:
from googletrans import Translator

text = "Hi?"

def translate(text):
    translator = Translator()

    try:
        translated = translator.translate(text, dest="pl_PL")
        return translated.text
    except:
        pass

print(translate(text))

Cześć?


In [5]:
from spylls.hunspell import Dictionary
import os
# Dictionary.from_files("pl_PL")
os.path

<module 'ntpath' from 'c:\\users\\yaroslav\\appdata\\local\\programs\\python\\python38\\lib\\ntpath.py'>